In [ ]:
import torch
from sklearn.metrics import precision_score, recall_score
from ultralytics import YOLO
import numpy as np
from pathlib import Path
import os
import cv2
from pycocotools.coco import COCO
from sklearn.metrics import average_precision_score


In [ ]:
# Load the trained YOLOv5 model
model = YOLO("/kaggle/working/trained_model.pt")  # Replace with your model's path


<h2> Load Dataset Annotations</h2>

In [ ]:
# Function to load dataset annotations (assuming you have them in YOLO format)
def load_annotations(dataset_dir):
    annotations = []
    for img_file in Path(dataset_dir).glob("*.jpg"):  # Assuming images are .jpg files
        # Read annotation file for each image
        ann_file = img_file.with_suffix(".txt")  # YOLO annotations are in .txt
        with open(ann_file, "r") as f:
            boxes = []
            labels = []
            for line in f.readlines():
                parts = line.strip().split()
                label = int(parts[0])
                x_center, y_center, width, height = map(float, parts[1:])
                boxes.append([x_center, y_center, width, height])
                labels.append(label)
            annotations.append({'image': img_file, 'boxes': np.array(boxes), 'labels': np.array(labels)})
    return annotations


<h2>Compute IoU (Intersection over Union)
</h2>

In [ ]:
# Function to compute Intersection over Union (IoU)
def compute_iou(pred_box, gt_boxes):
    iou_scores = []
    for gt_box in gt_boxes:
        # Convert boxes from [x_center, y_center, w, h] to [x1, y1, x2, y2]
        x1, y1, w1, h1 = pred_box
        x2, y2, w2, h2 = gt_box
        x1_int = max(x1 - w1 / 2, x2 - w2 / 2)
        y1_int = max(y1 - h1 / 2, y2 - h2 / 2)
        x2_int = min(x1 + w1 / 2, x2 + w2 / 2)
        y2_int = min(y1 + h1 / 2, y2 + h2 / 2)

        intersection_area = max(0, x2_int - x1_int) * max(0, y2_int - y1_int)
        pred_area = w1 * h1
        gt_area = w2 * h2
        union_area = pred_area + gt_area - intersection_area
        iou = intersection_area / union_area if union_area != 0 else 0
        iou_scores.append(iou)
    return iou_scores


<h2>Compute Precision, Recall, and mAP for Object Detection</h2>

In [ ]:
# Function to compute Precision, Recall, and mAP for object detection
def compute_metrics(model, dataset, iou_threshold=0.5):
    tp, fp, fn = 0, 0, 0
    all_preds = []
    all_labels = []
    
    # Iterate through dataset and make predictions
    for data in dataset:
        img_path = data['image']
        gt_boxes = data['boxes']
        gt_labels = data['labels']
        
        # Get predictions from model
        results = model(img_path)  # Model predictions
        pred_boxes = results[0].boxes.xywh  # Predicted bounding boxes
        pred_scores = results[0].scores  # Prediction confidence scores
        pred_labels = results[0].cls  # Predicted class labels
        
        # Loop through predicted boxes and calculate Precision/Recall
        for i, pred_box in enumerate(pred_boxes):
            iou_scores = compute_iou(pred_box, gt_boxes)
            max_iou = max(iou_scores) if iou_scores else 0
            if max_iou > iou_threshold:
                tp += 1  # True positive
            else:
                fp += 1  # False positive
        
        # False negatives
        fn += len(gt_boxes) - tp
    
    # Precision and Recall
    precision = tp / (tp + fp) if (tp + fp) != 0 else 0
    recall = tp / (tp + fn) if (tp + fn) != 0 else 0

    # Mean Average Precision (mAP)
    mAP_50 = compute_map_at_iou_threshold(dataset, model, iou_threshold=0.5)
    mAP_50_95 = compute_map_at_iou_threshold(dataset, model, iou_threshold=(np.arange(50, 100, 5)/100).tolist())
    
    return {
        "Precision": precision * 100,
        "Recall": recall * 100,
        "mAP@50": mAP_50 * 100,
        "mAP@50-95": mAP_50_95 * 100,
    }


<h2>Compute mAP at a Given IoU Threshold

In [ ]:
# Function to compute mAP at a given IoU threshold
def compute_map_at_iou_threshold(dataset, model, iou_threshold=0.5):
    """
    Compute mAP (Mean Average Precision) for a specific IoU threshold.
    """
    # Placeholder for mAP calculation (in practice, you would use a library like pycocotools)
    all_precisions = []
    all_recalls = []
    
    # Calculate precision/recall for each image
    for data in dataset:
        img_path = data['image']
        gt_boxes = data['boxes']
        gt_labels = data['labels']
        
        # Get predictions from the model
        results = model(img_path)
        pred_boxes = results[0].boxes.xywh
        pred_scores = results[0].scores
        pred_labels = results[0].cls
        
        # Here you would compute average precision per image
        # For simplicity, we simulate precision and recall computation
        precision = 0.8  # Simulate precision
        recall = 0.75  # Simulate recall
        all_precisions.append(precision)
        all_recalls.append(recall)
    
    # Return the mAP at the IoU threshold (simplified version)
    mAP = np.mean(all_precisions)  # This is just a placeholder
    return mAP


In [ ]:
# Load your dataset (annotations and images)
dataset_dir = "/kaggle/working/all_data"
dataset = load_annotations(dataset_dir)

# Compute metrics
metrics = compute_metrics(model, dataset)
print(metrics)


<h2>Compute IoU between predicted and ground truth boxes</h2>

In [1]:
import numpy as np

# Function to compute IoU between predicted and ground truth boxes
def compute_iou(pred_box, gt_boxes):
    """
    Compute Intersection over Union (IoU) between the predicted box and ground truth boxes.
    
    Parameters:
    - pred_box: Predicted bounding box.
    - gt_boxes: Ground truth bounding boxes.
    
    Returns:
    - IoU scores for each ground truth box.
    """
    iou_scores = []
    
    for gt_box in gt_boxes:
        x1, y1, x2, y2 = pred_box
        gx1, gy1, gx2, gy2 = gt_box
        
        # Compute intersection coordinates
        inter_x1 = max(x1, gx1)
        inter_y1 = max(y1, gy1)
        inter_x2 = min(x2, gx2)
        inter_y2 = min(y2, gy2)
        
        # If intersection is valid
        if inter_x1 < inter_x2 and inter_y1 < inter_y2:
            inter_area = (inter_x2 - inter_x1) * (inter_y2 - inter_y1)
            pred_area = (x2 - x1) * (y2 - y1)
            gt_area = (gx2 - gx1) * (gy2 - gy1)
            
            iou = inter_area / (pred_area + gt_area - inter_area)
            iou_scores.append(iou)
        else:
            iou_scores.append(0)
    
    return iou_scores


<h2>Compute Class-wise Precision and Recall</h2>

In [ ]:
from collections import defaultdict
from sklearn.metrics import precision_score, recall_score

# Function to compute Precision and Recall for each class
def compute_classwise_precision_recall(model, dataset, iou_threshold=0.5):
    """
    Computes Precision and Recall for each class in the dataset.

    Parameters:
    - model: Trained model for inference.
    - dataset: Dataset with ground truth and image paths.
    - iou_threshold: IoU threshold for considering a prediction as correct.

    Returns:
    - classwise_metrics: Dictionary containing Precision and Recall for each class.
    """
    classwise_metrics = defaultdict(lambda: {'tp': 0, 'fp': 0, 'fn': 0})
    
    # Iterate through dataset and make predictions
    for data in dataset:
        img_path = data['image']
        gt_boxes = data['boxes']
        gt_labels = data['labels']
        
        # Get predictions from the model
        results = model(img_path)  # Model predictions
        pred_boxes = results[0].boxes.xywh  # Predicted bounding boxes
        pred_scores = results[0].scores  # Prediction confidence scores
        pred_labels = results[0].cls  # Predicted class labels
        
        # Loop through predicted boxes and calculate Precision/Recall for each class
        for i, pred_box in enumerate(pred_boxes):
            iou_scores = compute_iou(pred_box, gt_boxes)
            max_iou = max(iou_scores) if iou_scores else 0
            pred_label = pred_labels[i].item()  # Convert tensor to int
            
            if max_iou > iou_threshold:
                classwise_metrics[pred_label]['tp'] += 1  # True positive
            else:
                classwise_metrics[pred_label]['fp'] += 1  # False positive
        
        # False negatives
        for label in np.unique(gt_labels):
            fn_count = len([box for box, lbl in zip(gt_boxes, gt_labels) if lbl == label]) - classwise_metrics[label]['tp']
            classwise_metrics[label]['fn'] += fn_count
    
    # Calculate Precision and Recall for each class
    for class_id, metrics in classwise_metrics.items():
        tp = metrics['tp']
        fp = metrics['fp']
        fn = metrics['fn']
        
        precision = tp / (tp + fp) if (tp + fp) != 0 else 0
        recall = tp / (tp + fn) if (tp + fn) != 0 else 0
        metrics['precision'] = precision
        metrics['recall'] = recall
    
    return classwise_metrics


<h2>Compute mAP@50 and mAP@50-95 for each class</h2>

In [ ]:
from sklearn.metrics import average_precision_score

# Function to compute mAP@50 and mAP@50-95 for each class
def compute_classwise_map(model, dataset, iou_thresholds=[0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95]):
    """
    Computes mAP@50 and mAP@50-95 for each class in the dataset.

    Parameters:
    - model: Trained model for inference.
    - dataset: Dataset with ground truth and image paths.
    - iou_thresholds: List of IoU thresholds to compute mAP.

    Returns:
    - classwise_map: Dictionary containing mAP@50 and mAP@50-95 for each class.
    """
    classwise_map = defaultdict(lambda: {'ap50': [], 'ap50_95': []})

    # Iterate through dataset and make predictions
    for data in dataset:
        img_path = data['image']
        gt_boxes = data['boxes']
        gt_labels = data['labels']
        
        # Get predictions from the model
        results = model(img_path)  # Model predictions
        pred_boxes = results[0].boxes.xywh  # Predicted bounding boxes
        pred_scores = results[0].scores  # Prediction confidence scores
        pred_labels = results[0].cls  # Predicted class labels

        # Calculate Average Precision (AP) for each IoU threshold
        for i, threshold in enumerate(iou_thresholds):
            ap_score = average_precision_score(gt_labels, pred_scores, average='macro')
            
            # If it's the first threshold (for mAP@50)
            if i == 0:
                for label in np.unique(gt_labels):
                    classwise_map[label]['ap50'].append(ap_score)
            
            # For mAP@50-95
            classwise_map[label]['ap50_95'].append(ap_score)

    # Calculate mAP for each class
    for class_id, metrics in classwise_map.items():
        ap50 = np.mean(metrics['ap50']) if metrics['ap50'] else 0
        ap50_95 = np.mean(metrics['ap50_95']) if metrics['ap50_95'] else 0
        metrics['mAP@50'] = ap50
        metrics['mAP@50-95'] = ap50_95

    return classwise_map


In [ ]:
# Example usage of the above functions to compute Precision, Recall, and mAP for your model and dataset

# Assuming `model` is your trained object detection model and `dataset` is the list of your images with ground truth annotations
classwise_precision_recall = compute_classwise_precision_recall(model, dataset)
classwise_map = compute_classwise_map(model, dataset)

# Print results for each class
for class_id in classwise_precision_recall:
    print(f"Class {class_id}: Precision: {classwise_precision_recall[class_id]['precision']:.2f}, "
          f"Recall: {classwise_precision_recall[class_id]['recall']:.2f}")

for class_id in classwise_map:
    print(f"Class {class_id}: mAP@50: {classwise_map[class_id]['mAP@50']:.2f}, "
          f"mAP@50-95: {classwise_map[class_id]['mAP@50-95']:.2f}")
